# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (112,112,1) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (16,16), strides=(16,16), input_shape=self.state_size, activation='relu'))
        model.add(Conv2D(64, (3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))       
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state, stochastic=False):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)[0]
        
        if stochastic:
            act_probs = np.exp(act_values)/np.exp(act_values).sum()
            return np.random.choice(np.arange(self.action_size), size=1, p=act_probs)[0]
              
        return np.argmax(act_values) 

    def replay(self): 
        if len(self.replay_buffer) < self.batch_size:
            return
        
        if len(self.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:    
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
        self.update_target_model()        
    
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame.mean(axis=2)
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

#### Test Suite

In [10]:
def test_agent(stochastic=False):
    current_epsilon = agent.epsilon
    agent.epsilon = 0.0
    num_solved = 0
    solved_in_steps = defaultdict(int)

    for t in tqdm(range(100)):    
        random.seed(t)
        sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
        sok.set_maxsteps(20)
        steps = 0

        state = sok.get_image('rgb_array')
        done = False
        while not done:
            steps += 1
            action = agent.act(process_frame(state), stochastic)
            if action < 4:
                action += 1
            else:
                action += 5
            state, reward, done, info = sok.step(action)

        if 3 in sok.room_state:            
            num_solved += 1
            solved_in_steps[steps] += 1
    
    agent.epsilon = current_epsilon
    print("*" * 30)
    print("Stochastic" if stochastic else "Deterministic")
    print("*" * 30)
    print("Solved: %d" % num_solved)
    print("=" * 30)
    print(solved_in_steps)
    print("*" * 30)

In [11]:
max_episodes = 50000
max_steps = 20

def init_sok(r):
    random.seed(r+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [ ]:
agent = SOK_Agent()

steps_per_episode = []

for e in range(max_episodes):
    if e % 100 == 0 and e > 0:
        test_agent(stochastic=False)
        
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    random.seed(e)
    
    state = process_frame(sok.get_image('rgb_array'))
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()                
        
        if (step+1) % agent.replay_rate == 0:
            agent.replay()            
        
        if done:   
            steps_per_episode.append(step+1)
            
            if 3 in sok.room_state:                
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(step+1)
            
            #agent.save("exp1_episode%d.h5" % (e))            
            break

Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 6 Epsilon 0.9322
Episode: 8
Episode: 9
Episode: 10
Episode: 11
Episode: 12
Episode: 13
Episode: 14
Episode: 15
Episode: 16
Episode: 17
Episode: 18
Episode: 19
Episode: 20
Episode: 21
Episode: 22
Episode: 23
Episode: 24
Episode: 25
Episode: 26
Episode: 27
Episode: 28
Episode: 29
SOLVED! Episode 29 Steps: 1 Epsilon 0.7553
Episode: 30
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39
Episode: 40
Episode: 41
Episode: 42
Episode: 43
Episode: 44
Episode: 45
Episode: 46
Episode: 47
Episode: 48
SOLVED! Episode 48 Steps: 1 Epsilon 0.6306
Episode: 49
Episode: 50
Episode: 51
Episode: 52
Episode: 53
Episode: 54
Episode: 55
Episode: 56
SOLVED! Episode 56 Steps: 14 Epsilon 0.5849
Episode: 57
Episode: 58
Episode: 59
Episode: 60
Episode: 61
SOLVED! Episode 61 Steps: 1 Epsilon 0.5619
Episode: 62
Episode: 63
Episode: 64
Episode: 65
SOL


******************************
Deterministic
******************************
Solved: 26
defaultdict(<class 'int'>, {3: 8, 1: 13, 2: 5})
******************************
Episode: 100
Episode: 101
Episode: 102
Episode: 103
Episode: 104
Episode: 105
Episode: 106
Episode: 107
Episode: 108
Episode: 109
Episode: 110
Episode: 111
Episode: 112
Episode: 113
Episode: 114
SOLVED! Episode 114 Steps: 1 Epsilon 0.3651
Episode: 115
Episode: 116
SOLVED! Episode 116 Steps: 3 Epsilon 0.3615
Episode: 117
Episode: 118
Episode: 119
Episode: 120
Episode: 121
Episode: 122
Episode: 123
SOLVED! Episode 123 Steps: 3 Epsilon 0.3404
Episode: 124
Episode: 125
SOLVED! Episode 125 Steps: 2 Epsilon 0.3370
Episode: 126
Episode: 127
Episode: 128
SOLVED! Episode 128 Steps: 2 Epsilon 0.3303
Episode: 129
SOLVED! Episode 129 Steps: 1 Epsilon 0.3303
Episode: 130
Episode: 131
Episode: 132
Episode: 133
SOLVED! Episode 133 Steps: 13 Epsilon 0.3189
Episode: 134
Episode: 135
Episode: 136
SOLVED! Episode 136 Steps: 2 Epsilon 0.3126


******************************
Deterministic
******************************
Solved: 40
defaultdict(<class 'int'>, {3: 16, 1: 17, 2: 7})
******************************
Episode: 200
Episode: 201
Episode: 202
Episode: 203
Episode: 204
Episode: 205
Episode: 206
Episode: 207
Episode: 208
Episode: 209
Episode: 210
Episode: 211
Episode: 212
SOLVED! Episode 212 Steps: 3 Epsilon 0.2988
Episode: 213
Episode: 214
SOLVED! Episode 214 Steps: 3 Epsilon 0.2988
Episode: 215
Episode: 216
SOLVED! Episode 216 Steps: 3 Epsilon 0.2988
Episode: 217
SOLVED! Episode 217 Steps: 3 Epsilon 0.2988
Episode: 218
Episode: 219
SOLVED! Episode 219 Steps: 2 Epsilon 0.2988
Episode: 220
SOLVED! Episode 220 Steps: 1 Epsilon 0.2988
Episode: 221
SOLVED! Episode 221 Steps: 1 Epsilon 0.2988
Episode: 222
Episode: 223
SOLVED! Episode 223 Steps: 1 Epsilon 0.2988
Episode: 224
Episode: 225
Episode: 226
Episode: 227
Episode: 228
Episode: 229
Episode: 230
Episode: 231
Episode: 232
SOLVED! Episode 232 Steps: 3 Epsilon 0.2988
Episode


******************************
Deterministic
******************************
Solved: 51
defaultdict(<class 'int'>, {3: 22, 1: 19, 2: 10})
******************************
Episode: 300
SOLVED! Episode 300 Steps: 1 Epsilon 0.2988
Episode: 301
Episode: 302
SOLVED! Episode 302 Steps: 2 Epsilon 0.2988
Episode: 303
SOLVED! Episode 303 Steps: 5 Epsilon 0.2988
Episode: 304
Episode: 305
SOLVED! Episode 305 Steps: 2 Epsilon 0.2988
Episode: 306
Episode: 307
SOLVED! Episode 307 Steps: 1 Epsilon 0.2988
Episode: 308
Episode: 309
Episode: 310
Episode: 311
Episode: 312
Episode: 313
Episode: 314
Episode: 315
SOLVED! Episode 315 Steps: 3 Epsilon 0.2988
Episode: 316
Episode: 317
Episode: 318
SOLVED! Episode 318 Steps: 2 Epsilon 0.2988
Episode: 319
Episode: 320
Episode: 321
Episode: 322
SOLVED! Episode 322 Steps: 1 Epsilon 0.2988
Episode: 323
Episode: 324
SOLVED! Episode 324 Steps: 1 Epsilon 0.2988
Episode: 325
Episode: 326
Episode: 327
SOLVED! Episode 327 Steps: 2 Epsilon 0.2988
Episode: 328
Episode: 329
E


******************************
Deterministic
******************************
Solved: 52
defaultdict(<class 'int'>, {3: 22, 1: 20, 2: 10})
******************************
Episode: 400
Episode: 401
Episode: 402
Episode: 403
SOLVED! Episode 403 Steps: 3 Epsilon 0.2988
Episode: 404
SOLVED! Episode 404 Steps: 3 Epsilon 0.2988
Episode: 405
SOLVED! Episode 405 Steps: 1 Epsilon 0.2988
Episode: 406
SOLVED! Episode 406 Steps: 1 Epsilon 0.2988
Episode: 407
Episode: 408
Episode: 409
Episode: 410
SOLVED! Episode 410 Steps: 2 Epsilon 0.2988
Episode: 411
Episode: 412
Episode: 413
SOLVED! Episode 413 Steps: 3 Epsilon 0.2988
Episode: 414
Episode: 415
SOLVED! Episode 415 Steps: 2 Epsilon 0.2988
Episode: 416
Episode: 417
Episode: 418
SOLVED! Episode 418 Steps: 3 Epsilon 0.2988
Episode: 419
Episode: 420
Episode: 421
SOLVED! Episode 421 Steps: 2 Epsilon 0.2988
Episode: 422
Episode: 423
SOLVED! Episode 423 Steps: 1 Epsilon 0.2988
Episode: 424
Episode: 425
Episode: 426
Episode: 427
Episode: 428
SOLVED! Episod

...


******************************
Deterministic
******************************
Solved: 77
defaultdict(<class 'int'>, {3: 25, 1: 21, 7: 2, 6: 9, 2: 11, 5: 7, 4: 2})
******************************
Episode: 12500
SOLVED! Episode 12500 Steps: 1 Epsilon 0.2988
Episode: 12501
SOLVED! Episode 12501 Steps: 3 Epsilon 0.2988
Episode: 12502
SOLVED! Episode 12502 Steps: 3 Epsilon 0.2988
Episode: 12503
SOLVED! Episode 12503 Steps: 3 Epsilon 0.2988
Episode: 12504
SOLVED! Episode 12504 Steps: 3 Epsilon 0.2988
Episode: 12505
SOLVED! Episode 12505 Steps: 6 Epsilon 0.2988
Episode: 12506
Episode: 12507
SOLVED! Episode 12507 Steps: 1 Epsilon 0.2988
Episode: 12508
SOLVED! Episode 12508 Steps: 2 Epsilon 0.2988
Episode: 12509
SOLVED! Episode 12509 Steps: 1 Epsilon 0.2988
Episode: 12510
SOLVED! Episode 12510 Steps: 8 Epsilon 0.2988
Episode: 12511
SOLVED! Episode 12511 Steps: 5 Epsilon 0.2988
Episode: 12512
Episode: 12513
SOLVED! Episode 12513 Steps: 3 Epsilon 0.2988
Episode: 12514
Episode: 12515
SOLVED! Episode


******************************
Deterministic
******************************
Solved: 80
defaultdict(<class 'int'>, {3: 25, 1: 21, 7: 2, 6: 7, 2: 11, 5: 13, 4: 1})
******************************
Episode: 12600
SOLVED! Episode 12600 Steps: 10 Epsilon 0.2988
Episode: 12601
SOLVED! Episode 12601 Steps: 3 Epsilon 0.2988
Episode: 12602
SOLVED! Episode 12602 Steps: 6 Epsilon 0.2988
Episode: 12603
SOLVED! Episode 12603 Steps: 3 Epsilon 0.2988
Episode: 12604
SOLVED! Episode 12604 Steps: 1 Epsilon 0.2988
Episode: 12605
SOLVED! Episode 12605 Steps: 1 Epsilon 0.2988
Episode: 12606
SOLVED! Episode 12606 Steps: 1 Epsilon 0.2988
Episode: 12607
SOLVED! Episode 12607 Steps: 4 Epsilon 0.2988
Episode: 12608
SOLVED! Episode 12608 Steps: 5 Epsilon 0.2988
Episode: 12609
SOLVED! Episode 12609 Steps: 2 Epsilon 0.2988
Episode: 12610
SOLVED! Episode 12610 Steps: 2 Epsilon 0.2988
Episode: 12611
SOLVED! Episode 12611 Steps: 5 Epsilon 0.2988
Episode: 12612
SOLVED! Episode 12612 Steps: 1 Epsilon 0.2988
Episode: 126


******************************
Deterministic
******************************
Solved: 79
defaultdict(<class 'int'>, {3: 25, 1: 21, 7: 1, 6: 7, 2: 11, 5: 13, 4: 1})
******************************
Episode: 12700
SOLVED! Episode 12700 Steps: 4 Epsilon 0.2988
Episode: 12701
SOLVED! Episode 12701 Steps: 3 Epsilon 0.2988
Episode: 12702
SOLVED! Episode 12702 Steps: 2 Epsilon 0.2988
Episode: 12703
SOLVED! Episode 12703 Steps: 2 Epsilon 0.2988
Episode: 12704
SOLVED! Episode 12704 Steps: 2 Epsilon 0.2988
Episode: 12705
Episode: 12706
SOLVED! Episode 12706 Steps: 3 Epsilon 0.2988
Episode: 12707
SOLVED! Episode 12707 Steps: 6 Epsilon 0.2988
Episode: 12708
Episode: 12709
SOLVED! Episode 12709 Steps: 5 Epsilon 0.2988
Episode: 12710
Episode: 12711
Episode: 12712
SOLVED! Episode 12712 Steps: 2 Epsilon 0.2988
Episode: 12713
SOLVED! Episode 12713 Steps: 3 Epsilon 0.2988
Episode: 12714
Episode: 12715
SOLVED! Episode 12715 Steps: 3 Epsilon 0.2988
Episode: 12716
SOLVED! Episode 12716 Steps: 6 Epsilon 0.2988


******************************
Deterministic
******************************
Solved: 83
defaultdict(<class 'int'>, {3: 25, 1: 21, 7: 2, 6: 10, 2: 11, 5: 13, 4: 1})
******************************
Episode: 12800
SOLVED! Episode 12800 Steps: 2 Epsilon 0.2988
Episode: 12801
Episode: 12802
SOLVED! Episode 12802 Steps: 4 Epsilon 0.2988
Episode: 12803
SOLVED! Episode 12803 Steps: 6 Epsilon 0.2988
Episode: 12804
SOLVED! Episode 12804 Steps: 3 Epsilon 0.2988
Episode: 12805
SOLVED! Episode 12805 Steps: 3 Epsilon 0.2988
Episode: 12806
SOLVED! Episode 12806 Steps: 7 Epsilon 0.2988
Episode: 12807
SOLVED! Episode 12807 Steps: 3 Epsilon 0.2988
Episode: 12808
SOLVED! Episode 12808 Steps: 1 Epsilon 0.2988
Episode: 12809
Episode: 12810
Episode: 12811
SOLVED! Episode 12811 Steps: 5 Epsilon 0.2988
Episode: 12812
Episode: 12813
SOLVED! Episode 12813 Steps: 2 Epsilon 0.2988
Episode: 12814
SOLVED! Episode 12814 Steps: 4 Epsilon 0.2988
Episode: 12815
SOLVED! Episode 12815 Steps: 6 Epsilon 0.2988
Episode: 1281


******************************
Deterministic
******************************
Solved: 80
defaultdict(<class 'int'>, {3: 25, 1: 20, 7: 2, 6: 9, 2: 11, 5: 11, 4: 2})
******************************
Episode: 12900
Episode: 12901
SOLVED! Episode 12901 Steps: 6 Epsilon 0.2988
Episode: 12902
SOLVED! Episode 12902 Steps: 1 Epsilon 0.2988
Episode: 12903
SOLVED! Episode 12903 Steps: 3 Epsilon 0.2988
Episode: 12904
SOLVED! Episode 12904 Steps: 6 Epsilon 0.2988
Episode: 12905
SOLVED! Episode 12905 Steps: 1 Epsilon 0.2988
Episode: 12906
SOLVED! Episode 12906 Steps: 5 Epsilon 0.2988
Episode: 12907
SOLVED! Episode 12907 Steps: 1 Epsilon 0.2988
Episode: 12908
Episode: 12909
SOLVED! Episode 12909 Steps: 2 Epsilon 0.2988
Episode: 12910
SOLVED! Episode 12910 Steps: 3 Epsilon 0.2988
Episode: 12911
SOLVED! Episode 12911 Steps: 2 Epsilon 0.2988
Episode: 12912
SOLVED! Episode 12912 Steps: 5 Epsilon 0.2988
Episode: 12913
SOLVED! Episode 12913 Steps: 5 Epsilon 0.2988
Episode: 12914
SOLVED! Episode 12914 Steps: 